<a href="https://colab.research.google.com/github/bunkorner/VR_Assignment2/blob/main/Assignment_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
srcPath = '/content/gdrive/MyDrive/CV/Assignment_2/Assignment2_BikeHorses'
destPath = '/content/gdrive/MyDrive/CV/Assignment_2/Bike_Horse'

In [4]:
if not os.path.exists(destPath):
    os.mkdir(destPath)

In [5]:
onlyDirs = [f for f in os.listdir(srcPath) if os.path.isdir(os.path.join(srcPath, f))]

for directory in onlyDirs:
    rdPath = srcPath+'/'+directory
    path = destPath+'/'+directory
    if not os.path.exists(path):
        os.makedirs(path)
        
    onlyfiles = [f for f in os.listdir(rdPath) if os.path.isfile(os.path.join(rdPath, f))]
    i=0
    for file in onlyfiles:
        i = i + 1
        imgPath = rdPath+'/'+file
        img = cv2.imread(imgPath)
        wrtPath = path+'/' + str(i) + '.jpg'
        cv2.imwrite(wrtPath,img)

In [6]:
imgs = []
labels = []
label = -1

dirs = [f for f in os.listdir(destPath) if os.path.isdir(os.path.join(destPath, f))]

for directory in dirs:
    label = label + 1
    path = destPath+'/'+directory
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    for file in files:
        imgPath = path+'/'+file
        imgs.append(imgPath)
        labels.append(label)

In [7]:
len(imgs)

188

In [8]:
imgFiles = []
for imgName in imgs:
    img = cv2.imread(imgName)
    imgFiles.append(img)
imgFiles = np.asarray(imgFiles)

<ipython-input-8-eb5b51ae2ef9>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  imgFiles = np.asarray(imgFiles)


In [9]:
from sklearn.cluster import KMeans
numberOfClusters=300

In [10]:
descriptors = np.asarray([])
ext = cv2.SIFT_create()
for img in imgFiles:
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    keypoints, desc = ext.detectAndCompute(gray, None)
    if type(desc) == np.ndarray :
        if descriptors.shape[0] == 0:
            descriptors = desc
        else:
            descriptors = np.concatenate((descriptors, desc), axis=0)
kmeans = KMeans(n_clusters = numberOfClusters)

In [11]:
descriptors.shape

(178044, 128)

In [12]:
kmeans.fit(descriptors)

KMeans(n_clusters=300)

In [13]:
allBOVW = []
ext = cv2.xfeatures2d.SIFT_create()
for img in imgFiles:
  bovw = [0]*numberOfClusters
  gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  keypoints, descriptors = ext.detectAndCompute(gray, None)
  if type(descriptors) == np.ndarray:
      pred = kmeans.predict(descriptors)
      for ele in pred:
          bovw[ele] = bovw[ele] + 1
  allBOVW.append(bovw)

In [14]:
allBOVW = np.asarray(allBOVW)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
trainImgs,testImgs, trainLabels, testLabels = train_test_split(allBOVW, labels, 
                                                      train_size=0.7, random_state=0)

In [17]:
trainImgs

array([[ 1,  0,  1, ...,  1,  2,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 5,  0, 16, ...,  5,  5,  1],
       ...,
       [ 1,  0,  6, ...,  0,  2,  3],
       [ 2,  0,  2, ...,  3,  4,  6],
       [ 0,  0,  3, ...,  3,  3,  2]])

In [20]:
from sklearn import svm
SVClassifier = svm.SVC(gamma=0.01,kernel='linear')
SVClassifier.fit(trainImgs,trainLabels)

SVC(gamma=0.01, kernel='linear')

In [21]:
SVClassifier.score(testImgs,testLabels)

0.9649122807017544